In [1]:
import os
import random

1.볼링 게임 클래스

- 초기화
- 보너스 점수 매서드
- 점수 계산 매서드
- 프레임 매서드

In [16]:
# 볼링 게임 클래스
class Bowling_game:
    def __init__(self):
        # 상수 선언
        self.defaultframe = 10
        self.defaultpin = 10
        self.strike = 2
        self.spare = 1
        self.normal = 0

        # 변수 선언
        self.throw1 = 0
        self.throw2 = 0
        self.throw3 = 0
        self.sum = 0
        self.total = 0

        # 볼링 점수를 저장할 딕셔너리
        self.result = {}
        for frames in range(self.defaultframe):
            self.result[str(frames + 1) + "프레임"] = [0, 0, 0, 0, 0, 0]

    # 범위를 구하는 함수
    def randpin(self, pins):
        return random.randint(0, pins)

    # 이전 프레임에 값을 누적하기 위한 함수
    def pastframecheck(self, tr, fr):
        for pastframes in range(fr - 2, fr):
            if pastframes < 0:
                continue  # 0보다 작을 경우, 동작하지 않음

            pastframe = str(pastframes + 1) + "프레임"

            if self.result[pastframe][5] > 0:  # 상태값이 0보다 클 때
                self.result[pastframe][3] += tr  # 이전 프레임의 점수에 이번 판 점수를 더함 => 프레임 점수 표기
                self.result[pastframe][4] += tr  # 지금까지의 점수에 이번 판 점수를 더함
                self.result[pastframe][5] -= 1  # 상태의 값에서 1을 빼주고 저장

    # 볼링 점수 계산
    def frame_score(self, tr1, tr2, tr3, fr, st):
        currentframe = str(fr + 1) + "프레임"
        self.result[currentframe][0] = tr1
        self.result[currentframe][1] = tr2
        self.result[currentframe][2] = tr3
        self.result[currentframe][3] = self.result[currentframe][0] + self.result[currentframe][1] + self.result[currentframe][2]
        self.result[currentframe][5] = st

        if fr > 0:
            self.result[currentframe][4] = self.result[currentframe][3] + self.result[str(fr) + "프레임"][4]  # 이전 프레임의 값을 더함
        else:
            self.result[currentframe][4] = self.result[currentframe][3]

    # 프레임 진행
    def play_game(self):

        for frames in range(self.defaultframe):
            self.throw1 = self.randpin(self.defaultpin)
            self.pastframecheck(self.throw1, frames)

            if self.throw1 == self.defaultpin:  # 만일 1투구 스트라이크
                self.frame_score(self.throw1, 0, 0, frames, self.strike)
            else:  # 1투구 스트라이크 실패
                leftpin = self.defaultpin - self.throw1
                self.throw2 = self.randpin(leftpin)

                if self.throw2 == leftpin:  # 2투구로 스페어
                    self.frame_score(self.throw1, self.throw2, 0, frames, self.spare)
                    self.pastframecheck(self.throw2, frames)
                else:  # 스페어 실패
                    self.frame_score(self.throw1, self.throw2, 0, frames, self.normal)
                    self.pastframecheck(self.throw2, frames)

            # 10프레임일때
            if frames == 9:
                self.throw1 = self.randpin(self.defaultpin)
                self.pastframecheck(self.throw1, frames)

                if self.throw1 == self.defaultpin:  # 10프레임 1투구 스트라이크
                    self.throw2 = self.randpin(self.defaultpin)
                    self.pastframecheck(self.throw2, frames)

                    if self.throw2 == self.defaultpin:  # 10프레임 2투구 스트라이크
                        self.throw3 = self.randpin(self.defaultpin)  # 3투구 마음껏
                        self.frame_score(self.throw1, self.throw2, self.throw3, frames, 0)

                    else:
                        leftpins = 10 - self.throw2  # 10프레임 2투구 스페어 실패
                        self.throw3 = self.randpin(leftpins)  # 점수 빼고 점수 계산
                        self.frame_score(self.throw1, self.throw2, self.throw3, frames, 0)

                else:  # 스트라이크 실패
                    leftpin = self.defaultpin - self.throw1
                    self.throw2 = self.randpin(leftpin)
                    self.pastframecheck(self.throw2, frames)

                    if self.throw2 == leftpin:  # 스페어 성공
                        self.throw3 = self.randpin(self.defaultpin)
                        self.frame_score(self.throw1, self.throw2, self.throw3, frames, 0)

                    else:
                        break
        return self.result

2. 파일입출력 클래스
- 초기화 ()
- 파일쓰기
- 파일읽기

In [13]:
class File_control:
    def __init__(self):
        pass #실제 사용되는 변수들은 없으나 선언이 필요함

    # 게임 결과 파일 저장
    def write_file(self, player, game_num, result):
        filepath = f"Bowling/{player}/game_{game_num}.txt"
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"{player}의 {game_num}번째 게임 결과\n")
            for frame, scores in result.items():
                file.write(f"{frame}: 투구1:{scores[0]}, 투구2:{scores[1]}, 투구3:{scores[2]}, 프레임점수:{scores[3]}, 누적점수:{scores[4]}\n")
            file.write(f"총점 :{scores[4]}\n")

    # 특정 게임 파일을 읽어서 출력
    def read_file(self, target_player, target_game):
        file_path = f"Bowling/{target_player}/game_{target_game}.txt"
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                print(f.read())


실행 예시

In [11]:
maxplayer = 10
random_player = random.randint(1, maxplayer)
p_list = [f"플레이어{index+1}" for index in range(random_player)]

In [17]:
file_control = File_control()

# 각 플레이어별 게임 플레이 및 결과 저장
for player in p_list:
    game_count = random.randint(1, 10)
    for game_num in range(1, game_count + 1):
        bowling_game = Bowling_game()
        result = bowling_game.play_game()
        file_control.write_file(player, game_num, result)

# 전체 결과 출력
for player in p_list:
    print(f"{player}의 전체 게임 결과")
    game_count = random.randint(1, 10)
    for game_num in range(1, game_count + 1):
        file_control.read_file(player, game_num)

# 원하는 게임 결과 출력
target_player = input("플레이어 이름을 입력하세요: ")
target_game = int(input("확인할 게임 번호를 입력하세요: "))
file_control.read_file(target_player, target_game)

플레이어1의 전체 게임 결과
플레이어1의 1번째 게임 결과
1프레임: 투구1:6, 투구2:0, 투구3:0, 프레임점수:6, 누적점수:6
2프레임: 투구1:2, 투구2:4, 투구3:0, 프레임점수:6, 누적점수:12
3프레임: 투구1:8, 투구2:1, 투구3:0, 프레임점수:9, 누적점수:21
4프레임: 투구1:1, 투구2:8, 투구3:0, 프레임점수:9, 누적점수:30
5프레임: 투구1:9, 투구2:0, 투구3:0, 프레임점수:9, 누적점수:39
6프레임: 투구1:10, 투구2:0, 투구3:0, 프레임점수:19, 누적점수:58
7프레임: 투구1:9, 투구2:0, 투구3:0, 프레임점수:9, 누적점수:67
8프레임: 투구1:0, 투구2:7, 투구3:0, 프레임점수:7, 누적점수:74
9프레임: 투구1:4, 투구2:1, 투구3:0, 프레임점수:5, 누적점수:79
10프레임: 투구1:10, 투구2:0, 투구3:0, 프레임점수:10, 누적점수:89
총점 :89

플레이어1의 2번째 게임 결과
1프레임: 투구1:9, 투구2:0, 투구3:0, 프레임점수:9, 누적점수:9
2프레임: 투구1:5, 투구2:4, 투구3:0, 프레임점수:9, 누적점수:18
3프레임: 투구1:7, 투구2:3, 투구3:0, 프레임점수:18, 누적점수:36
4프레임: 투구1:8, 투구2:0, 투구3:0, 프레임점수:8, 누적점수:44
5프레임: 투구1:7, 투구2:1, 투구3:0, 프레임점수:8, 누적점수:52
6프레임: 투구1:4, 투구2:2, 투구3:0, 프레임점수:6, 누적점수:58
7프레임: 투구1:4, 투구2:5, 투구3:0, 프레임점수:9, 누적점수:67
8프레임: 투구1:1, 투구2:6, 투구3:0, 프레임점수:7, 누적점수:74
9프레임: 투구1:10, 투구2:0, 투구3:0, 프레임점수:17, 누적점수:91
10프레임: 투구1:6, 투구2:1, 투구3:0, 프레임점수:7, 누적점수:97
총점 :97

플레이어1의 3번째 게임 결과
1프레임: 투구1:5, 투구2:3, 투구3:0, 프레임